# Создаём точку входа в Spark

# Самостоятельная работа

Требуется выяснить:
1. Какое соотношение сторон экрана телефона самое популярное,
2. Плотность пикселей у экрана.

Можно использовать только rdd.

Всего 8 баллов. 


## Считывание данных
Данные взяты отсюда: https://www.kaggle.com/iabhishekofficial/mobile-price-classification
Скачиваем и копируем в папку с нотебуком

Внутри содержится следующая информация:

* id: ID
* battery_power: Total energy a battery can store in one time (mAh)
* blue: Support bluetooth or not
* clock_speed: Speed at which microprocessor executes instructions
* dual_sim: Support dual sim or not
* fc: Front Camera mega pixels
* four_g: Support 4G or not
* int_memory: Internal Memory (GB)
* m_dep: Mobile Depth (cm)
* mobile_wt: Weight of mobile phone
* n_cores: Number of cores of processor
* pc: Primary Camera mega pixels
* px_height: Pixel Resolution Height
* px_width: Pixel Resolution Width
* ram: Random Access Memory (MB)
* sc_h: Screen Height of mobile (cm)
* sc_w: Screen Width of mobile (cm)
* talk_time: Time that a single battery charge will last
* three_g: Support 3G or not
* touch_screen: Has touch screen or not
* wifi: Support wifi or not

In [1]:
def beautiful_print(rdd, num, title):
    print(title)
    for row in rdd.take(num):
        print('\t', row)

In [5]:
from pyspark.sql import SparkSession

spark = SparkSession.builder\
        .master('local[4]')\
        .appName('Lesson_3')\
        .config('spark.ui.port', '4050')\
        .config('spark.executor.instances', 4)\
        .config('spark.executor.memory', f'{int(3000/1.1)}mb')\
        .config('spark.executor.cores', 2)\
        .getOrCreate()

sc = spark.sparkContext

In [106]:
train_rdd = sc.textFile('train.csv')
test_rdd = sc.textFile('test.csv')

In [53]:
# Выведите содержимое каждого файла (часть строк) (1 балл)

beautiful_print(train_rdd, 10, 'Train RDD')
beautiful_print(test_rdd, 10, 'Test RDD')

Train RDD
	 battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,pc,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,price_range
	 842,0,2.2,0,1,0,7,0.6,188,2,2,20,756,2549,9,7,19,0,0,1,1
	 1021,1,0.5,1,0,1,53,0.7,136,3,6,905,1988,2631,17,3,7,1,1,0,2
	 563,1,0.5,1,2,1,41,0.9,145,5,6,1263,1716,2603,11,2,9,1,1,0,2
	 615,1,2.5,0,0,0,10,0.8,131,6,9,1216,1786,2769,16,8,11,1,0,0,2
	 1821,1,1.2,0,13,1,44,0.6,141,2,14,1208,1212,1411,8,2,15,1,1,0,1
	 1859,0,0.5,1,3,0,22,0.7,164,1,7,1004,1654,1067,17,1,10,1,0,0,1
	 1821,0,1.7,0,4,1,10,0.8,139,8,10,381,1018,3220,13,8,18,1,0,1,3
	 1954,0,0.5,1,0,0,24,0.8,187,4,0,512,1149,700,16,3,5,1,1,1,0
	 1445,1,0.5,0,0,0,53,0.7,174,7,14,386,836,1099,17,1,20,1,0,0,0
Test RDD
	 id,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,pc,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi
	 1,1043,1,1.8,1,14,0,5,0.1,193,3,16,226,1412,3476,12,7,2,0,1,0
	 2,841,1,0.5

##  Преобразуем train и test

In [44]:
# train_header = train.first().split(',')
# test_header = test.first().split(',')
# train_header.insert(0, test_header[0]) # 'id' column
# test_header.insert(-1, train_header[-1]) # 'price_range' column


In [107]:
train_header = train_rdd.first()
test_header = test_rdd.first()
train_header, test_header

('battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,pc,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,price_range',
 'id,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,pc,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi')

In [108]:

train = train_rdd\
    .filter(lambda row: row != train_header)\
    .map(lambda row: [float(el) for el in row.split(',')])\
    .map(lambda row: [None] + row)

In [109]:
# Преобразуйте test (1 балл)

test = test_rdd\
    .filter(lambda row: row != test_header)\
    .map(lambda row: [float(el) for el in row.split(',')])\
    .map(lambda row: row + [None])

In [110]:
print('train_shape:', (train.count(), train.collect()[1].__len__()) )
print('test shape:', (test.count(), test.collect()[1].__len__()) )

train_shape: (2000, 22)
test shape: (1000, 22)


In [111]:
train_header = train_header.split(',')
test_header = test_header.split(',')
train_header.insert(0, test_header[0]) # 'id' column
test_header.insert(len(test_header), train_header[-1]) # 'price_range' column

In [112]:
n_cores = train_header.index('n_cores')

## Объединим train и test
Найти нужную функцию можно [здесь](https://spark.apache.org/docs/3.1.1/api/python/reference/pyspark.html#rdd-apis)

PS: нужно сделать средсвтвами rdd pd.concat([train, test,], axis=0)

In [113]:
train.collect().__len__(), test.collect().__len__()

(2000, 1000)

In [119]:
train.collect()[0][n_cores], test.collect()[-1][n_cores]

(2.0, 6.0)

In [129]:
# Объедините train и test (2 балла)
train_test_union = train.union(test)
train_test_union.collect().__len__()

3000

In [101]:
# объединение успешно
train_test_union.first()[n_cores], train_test_union.collect()[-1][n_cores]

(2.0, 6.0)

## Рассчитайте соотношение сторон телефона и экрана

In [194]:
# Выведите отсортированное распределение соотношений сторон экрана(1 балла)
# в разрезе широкоформатные или нет (экран широкоформатный, если соотногшение >=16:9)
# sc_h, sc_w

height = train_header.index('sc_h')
width = train_header.index('sc_w')

# screen_wh_relation = train_test_union.map(lambda row: (row[height], row[width]))\
#                                      .reduceByKey(lambda x, y: print(x, y))\
#                                      .sortByKey()\
#                                      .values()\
#                                      .collect()

# for row in train_test_union.collect():
#     # print(row[height], row[width])
#     if row[width] == 0:
#         print(f'WARN: [{row[height], row[width]}]')
wideformat = (16, 9)

train_test_union.map(lambda row: (row[height], row[width]))\
    .filter(lambda x: 0.0 not in x)\
    .map(lambda x: (x, 1) if (x[0] >= wideformat[0] and x[1] >= wideformat[1]) or (x[1]/x[0] >= wideformat[1]/wideformat[0]) else (x, 0))\
    .sortByKey(ascending=False)\
    .collect()

[((19.0, 18.0), 1),
 ((19.0, 18.0), 1),
 ((19.0, 18.0), 1),
 ((19.0, 18.0), 1),
 ((19.0, 18.0), 1),
 ((19.0, 18.0), 1),
 ((19.0, 18.0), 1),
 ((19.0, 18.0), 1),
 ((19.0, 18.0), 1),
 ((19.0, 18.0), 1),
 ((19.0, 18.0), 1),
 ((19.0, 18.0), 1),
 ((19.0, 18.0), 1),
 ((19.0, 17.0), 1),
 ((19.0, 17.0), 1),
 ((19.0, 17.0), 1),
 ((19.0, 17.0), 1),
 ((19.0, 17.0), 1),
 ((19.0, 17.0), 1),
 ((19.0, 17.0), 1),
 ((19.0, 17.0), 1),
 ((19.0, 17.0), 1),
 ((19.0, 17.0), 1),
 ((19.0, 17.0), 1),
 ((19.0, 17.0), 1),
 ((19.0, 17.0), 1),
 ((19.0, 17.0), 1),
 ((19.0, 17.0), 1),
 ((19.0, 16.0), 1),
 ((19.0, 16.0), 1),
 ((19.0, 16.0), 1),
 ((19.0, 16.0), 1),
 ((19.0, 16.0), 1),
 ((19.0, 16.0), 1),
 ((19.0, 16.0), 1),
 ((19.0, 16.0), 1),
 ((19.0, 15.0), 1),
 ((19.0, 15.0), 1),
 ((19.0, 15.0), 1),
 ((19.0, 15.0), 1),
 ((19.0, 15.0), 1),
 ((19.0, 15.0), 1),
 ((19.0, 15.0), 1),
 ((19.0, 15.0), 1),
 ((19.0, 15.0), 1),
 ((19.0, 15.0), 1),
 ((19.0, 15.0), 1),
 ((19.0, 14.0), 1),
 ((19.0, 14.0), 1),
 ((19.0, 14.0), 1),


In [286]:
# Выведите отсортированное распределение плотности пикселей (1 балл)
# px_height, px_width, 1in ~ 96px

height = train_header.index('px_height')
width = train_header.index('px_width')
# как-то здесь не так. но я не понимаю, откуда взять диагональ
# перевести пиксели высоты и ширины в дюймы и высчитать по теореме Пифагора?

print('(px_height, px_width), PPI')
train_test_union.map(lambda row: (row[width], row[height]))\
    .map(lambda x: (x, (x[0]**2 + x[1]**2)**0.5/((x[1]/64)**2 + (x[0]/64)**2)))\
    .sortByKey(ascending=False)\
    .collect()

(px_height, px_width), PPI


[((1998.0, 186.0), 2.041224175757678),
 ((1998.0, 68.0), 2.0488637781621244),
 ((1997.0, 1341.0), 1.702785133983266),
 ((1997.0, 819.0), 1.89768582188049),
 ((1996.0, 975.0), 1.843877931849261),
 ((1996.0, 521.0), 1.985577508466498),
 ((1995.0, 1738.0), 1.5480685076537963),
 ((1995.0, 777.0), 1.9131512109374929),
 ((1995.0, 517.0), 1.987479983234612),
 ((1994.0, 1949.0), 1.4689935453886338),
 ((1994.0, 224.0), 2.0413225282413863),
 ((1993.0, 849.0), 1.8907826631417912),
 ((1992.0, 1463.0), 1.6572767612543096),
 ((1991.0, 1308.0), 1.719408698696534),
 ((1990.0, 177.0), 2.0501977084862277),
 ((1989.0, 1210.0), 1.7593467809549177),
 ((1989.0, 1125.0), 1.7924707464279632),
 ((1989.0, 494.0), 1.998605994849919),
 ((1989.0, 401.0), 2.018708578423291),
 ((1988.0, 1790.0), 1.5311487284082987),
 ((1988.0, 1475.0), 1.6546599505707902),
 ((1988.0, 1446.0), 1.666216745547145),
 ((1988.0, 905.0), 1.8751997997505554),
 ((1988.0, 809.0), 1.9083960891072846),
 ((1987.0, 1197.0), 1.7657505074952458),
 

# JOIN
Повторите вышеописанное задание с помощью одной из функций ниже (отдельно рассчитайте для train и test, затем объедините результат)

Пример для двух RDDs: (rdd = {(1, 2), (3, 4), (3, 6)} other = {(3, 9)})

Имя функции |	Purpose |	Example |	Result
------------- |	------- |	------- |	------
subtractByKey |Remove elements with a key present in the other RDD.| rdd.subtractByKey(other) | {(1, 2)}
join | Perform an inner join between two RDDs. | rdd.join(other) | {(3, (4, 9)), (3, (6, 9))}
rightOuterJoin | Perform a join between two RDDs where the key must be present in the first RDD. | rdd.rightOuterJoin(other) | {(3,(Some(4),9)), (3,(Some(6),9))}leftOuterJoin | Perform a join between two RDDs where the key must be present in the other RDD. | rdd.leftOuterJoin(other) | {(1,(2,None)), (3,(4,Some(9))), (3,(6,Some(9)))}
cogroup | Group data from both RDDs sharing the same key. | rdd.cogroup(other) | {(1,([2],[])), (3,([4, 6],[9]))}


In [ ]:
# 2 балла
###############
# Ваш код здесь
###############

# DataFrame
Теперь мы знаем про Dataframe. Нужно сделать практически всё то же самое, но используя датафрейм.

In [250]:
# Считываем и объединяем данные (1 балл)
# Приведите все данные к правильному типу, либо считайе сразу верно (1 балл)

train_df = spark.read.csv('train.csv', header=True, inferSchema=True)
test_df = spark.read.csv('test.csv', header=True, inferSchema=True)

union_df = train_df.unionByName(test_df, allowMissingColumns=True)
union_df.show(n=1, vertical=True)

-RECORD 0-------------
 battery_power | 842  
 blue          | 0    
 clock_speed   | 2.2  
 dual_sim      | 0    
 fc            | 1    
 four_g        | 0    
 int_memory    | 7    
 m_dep         | 0.6  
 mobile_wt     | 188  
 n_cores       | 2    
 pc            | 2    
 px_height     | 20   
 px_width      | 756  
 ram           | 2549 
 sc_h          | 9    
 sc_w          | 7    
 talk_time     | 19   
 three_g       | 0    
 touch_screen  | 0    
 wifi          | 1    
 price_range   | 1    
 id            | null 
only showing top 1 row



In [291]:
# Создаём колонки с соотношением сторон и плотностью пикселей (1 балл)
from pyspark.sql import functions as f

union_df.selectExpr('sc_h', 'sc_w', 
                    'IF((sc_h/sc_w >= 16/9) OR (sc_h >= 16 AND sc_w >= 9), 1, 0) AS is_wideformat', 
                    'px_height', 'px_width', '(SQRT(POW(sc_h, 2) + POW(sc_w, 2))) * 0.4 AS diag',
                    'ROUND(SQRT(POW(px_width, 2) + POW(px_height, 2)) / (SQRT(POW(sc_h, 2) + POW(sc_w, 2))) * 0.4) AS PPI')\
                    .orderBy(['px_height'], ascending=False)\
                    .collect()

[Row(sc_h=18, sc_w=17, is_wideformat=1, px_height=1960, px_width=1963, diag=9.903534722511958, PPI=45.0),
 Row(sc_h=17, sc_w=8, is_wideformat=1, px_height=1949, px_width=1994, diag=7.515317691222375, PPI=59.0),
 Row(sc_h=6, sc_w=1, is_wideformat=1, px_height=1920, px_width=1933, diag=2.433105012119288, PPI=179.0),
 Row(sc_h=6, sc_w=3, is_wideformat=1, px_height=1914, px_width=1928, diag=2.683281572999748, PPI=162.0),
 Row(sc_h=17, sc_w=12, is_wideformat=1, px_height=1907, px_width=1911, diag=8.323460818673926, PPI=52.0),
 Row(sc_h=17, sc_w=3, is_wideformat=1, px_height=1901, px_width=1920, diag=6.905070600652827, PPI=63.0),
 Row(sc_h=5, sc_w=2, is_wideformat=1, px_height=1900, px_width=1959, diag=2.1540659228538015, PPI=203.0),
 Row(sc_h=9, sc_w=3, is_wideformat=1, px_height=1899, px_width=1904, diag=3.7947331922020555, PPI=113.0),
 Row(sc_h=13, sc_w=11, is_wideformat=0, px_height=1895, px_width=1976, diag=6.811754546370562, PPI=64.0),
 Row(sc_h=16, sc_w=15, is_wideformat=1, px_height=

In [336]:
new_union_df = union_df.withColumn('PPI', (union_df.px_height**2 + union_df.px_width**2)**0.5/(((union_df.sc_h**2 + union_df.sc_w**2)**0.5) / 2.54))

## Сохранение


In [360]:
from pyspark.sql.functions import udf,col
from pyspark.sql.types import StringType

# Сохраните результат в csv sep=';', encoding='cp1251'
# с колонками id, плотность пикселей и временем разговора в формате "1day 1hour 1minute"
# 2 балла
def time_from_minutes(time):
    day, hour, minute = 0, 0, 0
    day = time*60 // (24 * 3600)
    time = time*60 % (24*3600)
    hour = time // 3600
    time %= 3600
    minute = time // 60
    return f'{day}day {hour}hour {minute}minute'


udf_time = udf(lambda x:time_from_minutes(x),StringType() )


df_to_csv = new_union_df.withColumn('talk_time', udf_time(col('talk_time'))).selectExpr('id', 'ROUND(PPI, 2) PPI', 'talk_time')

df_to_csv.coalesce(1).write.csv('result', sep=';', encoding='cp1251')